# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
city_weather_csv = "../output_data/City_Weather.csv"
city_weather = pd.read_csv(city_weather_csv)
city_weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,97,NZ,1587503384,95,-46.19,168.86,283.15,0.89
1,ushuaia,75,AR,1587503374,81,-54.80,-68.30,279.15,13.90
2,zykovo,100,RU,1587503384,51,55.95,93.15,273.15,2.00
3,yerbogachen,97,RU,1587503384,95,61.28,108.01,271.50,2.62
4,airai,16,TL,1587503384,70,-8.93,125.41,288.77,1.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
locations = city_weather[["Lat","Lng"]]
weights = city_weather["Humidity"]
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
city_weather["Max Temp"] = (city_weather["Max Temp"] - 273.15) * 9/5 + 32

In [11]:
weather_filter = city_weather[city_weather["Max Temp"] > 70].copy()
weather_filter2 = weather_filter[weather_filter["Max Temp"] < 80].copy()
weather_filter3 = weather_filter2[weather_filter2["Wind Speed"] < 10].copy()
weather_filter4 = weather_filter3[weather_filter3["Cloudiness"] == 0].copy()
vacation_spots = weather_filter4.copy()
vacation_spots

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
267,betul,0,IN,1587503945,25,21.92,77.90,78.692,1.68
270,sao filipe,0,CV,1587503513,78,14.90,-24.50,72.716,7.83
273,pisco,0,PE,1587503497,64,-13.70,-76.22,75.200,7.70
281,cidreira,0,BR,1587503521,74,-30.18,-50.21,70.646,2.52
301,jalu,0,LY,1587503268,37,29.03,21.55,75.650,4.34
315,atar,0,MR,1587504019,16,20.52,-13.05,78.242,4.43
327,baghdad,0,IQ,1587503452,38,33.34,44.40,75.200,3.60
352,inhambane,0,MZ,1587504090,83,-23.86,35.38,73.598,4.17
386,mmathubudukwane,0,BW,1587504160,31,-24.60,26.43,73.400,2.10
517,pomerode,0,BR,1587504379,73,-26.74,-49.18,71.006,0.89


In [12]:
vacation_spots["Hotel Name"] = ""
vacation_spots

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
267,betul,0,IN,1587503945,25,21.92,77.90,78.692,1.68,
270,sao filipe,0,CV,1587503513,78,14.90,-24.50,72.716,7.83,
273,pisco,0,PE,1587503497,64,-13.70,-76.22,75.200,7.70,
281,cidreira,0,BR,1587503521,74,-30.18,-50.21,70.646,2.52,
301,jalu,0,LY,1587503268,37,29.03,21.55,75.650,4.34,
315,atar,0,MR,1587504019,16,20.52,-13.05,78.242,4.43,
327,baghdad,0,IQ,1587503452,38,33.34,44.40,75.200,3.60,
352,inhambane,0,MZ,1587504090,83,-23.86,35.38,73.598,4.17,
386,mmathubudukwane,0,BW,1587504160,31,-24.60,26.43,73.400,2.10,
517,pomerode,0,BR,1587504379,73,-26.74,-49.18,71.006,0.89,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-13.70,-76.22",
    "rankby": "distance",
    "type": "lodging",
    #"radius" : "5000",
    "key": g_key,
}
for index, row in vacation_spots.iterrows():
    params.update({"location" : f"{row['Lat']},{row['Lng']}"})
    response = requests.get(base_url, params=params).json()
    results = response['results']
    vacation_spots.loc[index, 'Hotel Name'] = results[0]['name']

In [33]:
vacation_spots

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
267,betul,0,IN,1587503945,25,21.92,77.90,78.692,1.68,HOTEL SONI
270,sao filipe,0,CV,1587503513,78,14.90,-24.50,72.716,7.83,Pensão Ngozi Nova Cidade
273,pisco,0,PE,1587503497,64,-13.70,-76.22,75.200,7.70,Hotel Presidente
281,cidreira,0,BR,1587503521,74,-30.18,-50.21,70.646,2.52,Pousada 591
301,jalu,0,LY,1587503268,37,29.03,21.55,75.650,4.34,Jalu Hotel
315,atar,0,MR,1587504019,16,20.52,-13.05,78.242,4.43,Oumou Elghoura
327,baghdad,0,IQ,1587503452,38,33.34,44.40,75.200,3.60,فندق البشائر
352,inhambane,0,MZ,1587504090,83,-23.86,35.38,73.598,4.17,Hotel Casa Do Capitão
386,mmathubudukwane,0,BW,1587504160,31,-24.60,26.43,73.400,2.10,Gill Sleepover Guesthouse
517,pomerode,0,BR,1587504379,73,-26.74,-49.18,71.006,0.89,Pousada Lena Rosa


In [34]:
narrowed_city_df = vacation_spots.copy()
hotel_df = vacation_spots.copy()

In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))